In [17]:
%matplotlib widget

import pandas
import numpy as np
import scipy
from scipy import stats
import matplotlib.pyplot as plt

calibration_data = pandas.read_feather('data_dump.bin')
test_data = pandas.read_feather('data_dump_2.bin')

In [18]:
calibration_data.describe()

,x,y,z
count,275291.000000,275291.000000,275291.000000
mean,0.107415,-0.028098,-0.810965
std,0.042703,0.031004,0.084228
min,0.000000,-0.184000,-1.120000
25%,0.072000,-0.040000,-0.880000
50%,0.104000,-0.024000,-0.840000
75%,0.144000,0.000000,-0.752000
max,0.328000,0.120000,-0.424000


In [19]:
test_data.describe()

,x,y,z
count,3815.000000,3815.000000,3815.000000
mean,0.102990,-0.039633,-0.818841
std,0.111913,0.113110,0.087799
min,-0.696000,-0.672000,-1.064000
25%,0.056000,-0.072000,-0.880000
50%,0.104000,-0.032000,-0.848000
75%,0.152000,0.000000,-0.760000
max,2.044000,0.728000,-0.504000


In [20]:
plt.plot(test_data['x'], test_data['y'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
rot_x_dps = np.arctan2(calibration_data['x'], np.hypot(calibration_data['y'], calibration_data['z']))
avg_rot_x = np.average(rot_x_dps)

rot_y_dps = np.arctan2(calibration_data['y'], np.hypot(calibration_data['x'], calibration_data['z']))
avg_rot_y = np.average(rot_y_dps)


pitch = avg_rot_y
roll = avg_rot_x

print(pitch)
print(roll)

-0.036099220064138654
0.1357483111992603


In [22]:
def get_rotation_matrix(yaw, pitch, roll):
    return np.array([
        [np.cos(roll) * np.cos(pitch), np.cos(pitch) * np.sin(roll), -np.sin(pitch)],
        [np.cos(roll) * np.sin(yaw) * np.sin(pitch) - np.cos(yaw) * np.sin(roll),
         np.cos(yaw) * np.cos(roll) + np.sin(yaw) * np.sin(roll) * np.sin(pitch), np.cos(pitch) * np.sin(yaw)],
        [np.sin(yaw) * np.sin(roll) + np.cos(yaw) * np.cos(roll) * np.sin(pitch),
         np.cos(yaw) * np.sin(roll) * np.sin(pitch) - np.cos(roll) * np.sin(yaw), np.cos(yaw) * np.cos(pitch)]])


# Attempt to remove rotation from the dataframe
rot_removed_df = calibration_data @ get_rotation_matrix(0, -pitch, -roll)
rot_removed_df.columns = ['x', 'y', 'z']

print(rot_removed_df)

               x         y         z
0       0.102929 -0.054430 -0.740429
1       0.030084 -0.028332 -0.841763
2       0.073348 -0.026167 -0.771253
3       0.120426 -0.048746 -0.725017
4       0.039088 -0.021488 -0.842051
...          ...       ...       ...
275286  0.039369 -0.069972 -0.874319
275287  0.066891 -0.041434 -0.891175
275288  0.070189 -0.049958 -0.547110
275289  0.028205 -0.044224 -0.833768
275290  0.062218 -0.016573 -0.890886

[275291 rows x 3 columns]


In [23]:
# Remove gravity from calibration matrix and compute gains
calibration_gains = np.average(rot_removed_df + np.array([0, 0, 1]))
calibration_std = np.array(np.std(rot_removed_df))
print(calibration_gains)
print(f"Calibration STD: {calibration_std}")

0.07361223946973729
Calibration STD: [0.04196746 0.03433899 0.08329804]


In [24]:
# Stupid me did not record time - so guess a dt // that will only really scale it
dt = 1/500

v_dps = []
p_dps = []

v = np.zeros(3, dtype=np.float64)
p = np.zeros(3, dtype=np.float64)

# Perform semi-implicit euler integration using 2nd derivative/acceleration
for i in range(len(test_data)):
    measurement = test_data.loc[i] - calibration_gains
    #print(measurement)
    v += measurement * dt
    p += v * dt
    
    v_dps.append(v.copy())
    p_dps.append(p.copy())

# Turn integrated points into NumPy arrays
v_np = np.array(v_dps)
p_np = np.array(p_dps)

#print(p_np)
#print(p_np[:, 0], p_np[:, 1])

pos_df = pandas.DataFrame(p_np)

plt.scatter(p_np[:, 0], p_np[:, 1])

pos_df.describe()

,0,1,2
count,3.815000e+03,3.815000e+03,3815.000000
mean,2.675426e-01,-1.086836e+00,-8.641999
std,2.429428e-01,9.760547e-01,7.737258
min,-1.793469e-07,-3.273613e+00,-25.947668
25%,4.856735e-02,-1.834469e+00,-14.579416
50%,1.945287e-01,-8.099621e-01,-6.475903
75%,4.501105e-01,-2.016960e-01,-1.616290
max,8.198489e-01,-3.584490e-07,-0.000004


In [25]:
%matplotlib widget

# Plot a vector field showing velocity in all points
plt.quiver(p_np[:, 0], p_np[:, 1], v_np[:, 0], v_np[:, 1], scale=1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
# First measurement data processing - gets rid of gravity, rotation and applies calibration gains
p1_measurement_data = test_data @ get_rotation_matrix(0, -pitch, -roll) - calibration_gains + [0, 0, 1]
p1_measurement_data.columns = ['x', 'y', 'z']

p1_measurement_data.describe()

,x,y,z
count,3815.000000,3815.000000,3815.000000
mean,-0.006281,-0.122810,0.104363
std,0.110757,0.115105,0.086664
min,-0.813615,-0.743709,-0.139518
25%,-0.051062,-0.156346,0.041132
50%,-0.007293,-0.116675,0.077028
75%,0.039317,-0.081509,0.162741
max,1.969210,0.627482,0.417519


In [54]:
# Filter out DPs that are less than 2 stdevs from a 0 value. Also include DP number 

NUM_STDEVS = 0
p2_measurement_data = p1_measurement_data[np.logical_and(np.abs(p1_measurement_data['x']) > NUM_STDEVS*calibration_std[0], np.abs(p1_measurement_data['y']) > NUM_STDEVS*calibration_std[1])]
p2_measurement_data.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [55]:
# Filter out datapoints that 
filtered_data = p2_measurement_data
filtered_data

,x,y,z
0,-0.067354,-0.090616,0.053512
1,-0.009740,-0.082337,0.051491
2,-0.022564,-0.112882,0.035790
3,0.007977,-0.117054,0.002656
4,0.047672,-0.122475,0.225355
...,...,...,...
3810,-0.064616,-0.074841,0.069502
3811,-0.024438,-0.080329,0.084048
3812,0.020251,-0.126804,0.154268
3813,0.035331,-0.153087,0.001501


In [78]:
v_dps = []
p_dps = []

v = np.zeros(3, dtype=np.float64)
p = np.zeros(3, dtype=np.float64)

# Perform semi-implicit euler integration using 2nd derivative/acceleration
last_i = 0

DT_PER_I = 1/100

for i in filtered_data.index:
    measurement = filtered_data.loc[i]
    
    # Compute dt
    dt = (i - last_i) * DT_PER_I
    last_i = i
    
    #print(measurement)
    #v += (measurement - -0.006280741567885446) * dt
    v += measurement * dt
    p += v * dt
    
    v_dps.append(v.copy())
    p_dps.append(p.copy())

# Turn integrated points into NumPy arrays
v_np = np.array(v_dps)
p_np = np.array(p_dps)

p_np

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-9.73953079e-07, -8.23365069e-06,  5.14911013e-06],
       [-4.20425974e-06, -2.77555034e-05,  1.38772459e-05],
       ...,
       [-5.30752831e+00, -8.85640991e+01,  7.68243828e+01],
       [-5.30991418e+00, -8.86109342e+01,  7.68641906e+01],
       [-5.31230355e+00, -8.86577771e+01,  7.69039997e+01]])

In [79]:
%matplotlib widget

pos_df = pandas.DataFrame(p_np)
plt.scatter(p_np[:, 0], p_np[:, 1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [80]:
%matplotlib widget

plt.scatter(v_np[:, 0], v_np[:, 1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
%matplotlib widget
filtered_data_np = np.array(filtered_data)

axis = 0

plt.subplot(3, 1, 1)
plt.title("Acceleration")
plt.plot(filtered_data_np[:, axis])

plt.subplot(3, 1, 2)
plt.title("Velocity")
plt.plot(v_np[:, axis])

plt.subplot(3, 1, 3)
plt.title("Position")
plt.plot(p_np[:, axis])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [82]:
np.average(filtered_data_np[:, 0])

-0.006280741567885446

In [74]:
# NOTE: I am aware accel data is still in g's, but that only really scales things

In [180]:
%matplotlib widget

# Just work with x datapoints to make life easier (assume 100Hz const sample rate )
t_dps = np.array(list(range(len(filtered_data_np)))) / 100
ax_dps = filtered_data_np[:, 0]

# Get the samples that almost certainly lie within the noise (-> deviate less than 1.8 STD)
noise_thres = 1.8 * calibration_std[0]
noise_thres_dps = ax_dps[np.abs(ax_dps) < noise_thres]
noise_avg = np.average(noise_thres_dps)

plt.plot(t_dps, ax_dps, zorder=1)
plt.axhline(noise_avg, np.min(t_dps), np.max(t_dps), color='orange')
plt.fill_between(t_dps, -noise_thres, +noise_thres, color='orange', alpha=0.5, zorder=2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [181]:
%matplotlib widget

# Based on: https://realpython.com/python-scipy-fft/
from scipy.fft import fft, fftfreq, irfft
SAMPLE_RATE = 100
DURATION = len(t_dps) / SAMPLE_RATE
N = int(SAMPLE_RATE * DURATION)
yf = fft(ax_dps)
xf = fftfreq(N, 1 / SAMPLE_RATE)

plt.plot(xf, np.abs(yf))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [182]:
# Similar to last one, but now on the calibration data set's x dps
%matplotlib widget

SAMPLE_RATE = 500
DURATION = len(rot_removed_df) / SAMPLE_RATE
N = int(SAMPLE_RATE * DURATION)
yf = fft(np.array(rot_removed_df)[:, 0])
xf = fftfreq(N, 1 / SAMPLE_RATE)

plt.plot(xf, np.abs(yf))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [183]:
%matplotlib widget
from scipy import signal
sos_filter = signal.butter(10, 0.0001, 'lp', fs=1/SAMPLE_RATE, output='sos')
filtered = signal.sosfilt(sos_filter, ax_dps)


plt.plot(t_dps, filtered, zorder=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [184]:
# NOTE: Because we end stationary/don't rotate, the sum of all acceleration (times the timestep, which is assumed constant -> can drop out) should be 0 -> we can just average the entire dataset under these idealized assumptions
filtered_normed = filtered - np.average(filtered)

In [185]:
np.std(filtered_normed)

0.040139630970043534

In [186]:
# Run another FFT on the filtered data to see what kind of difference it made
%matplotlib widget

# Based on: https://realpython.com/python-scipy-fft/
from scipy.fft import fft, fftfreq
SAMPLE_RATE = 100
DURATION = len(filtered_normed) / SAMPLE_RATE
N = int(SAMPLE_RATE * DURATION)
yf = fft(filtered_normed)
xf = fftfreq(N, 1 / SAMPLE_RATE)

plt.plot(xf, np.abs(yf))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [187]:
%matplotlib widget
dt = 1/SAMPLE_RATE
x = 0
vx = 0

t_dps = []
x_dps = []
vx_dps = []

for i in range(len(filtered_normed)):
    vx += filtered_normed[i] * dt
    x += vx * dt
    
    t_dps.append(i * dt)
    x_dps.append(x)
    vx_dps.append(vx)

plt.plot(t_dps, x_dps)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [203]:
%matplotlib widget
# NOTE: FFT appears to change magnitude as well, or at least, it appears to do so when filtering frequencies
from scipy.fft import ifft
signal_thres = 13
new_filtered_signal = ifft(yf[np.abs(yf) > signal_thres])
plt.plot(new_filtered_signal.real)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [208]:
f2_dps = new_filtered_signal.real

In [211]:
%matplotlib widget
f3_dps = signal.sosfilt(sos_filter, f2_dps)
plt.plot(f3_dps)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [212]:
%matplotlib widget
plt.plot(f2_dps)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [213]:
np.average(f2_dps)

0.4479926712251932

In [217]:
%matplotlib widget
plt.plot(t_dps, ax_dps)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [233]:
%matplotlib widget
import scipy.signal
savgol_filtered = signal.savgol_filter(ax_dps, 51, 10, deriv=0, delta=1.0, axis=- 1, mode='interp', cval=0.0)


plt.plot(t_dps, ax_dps)
plt.plot(t_dps, savgol_filtered)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …